In [71]:
import lab2_setup
import numpy as np
import sympy
import time

#####################
Qc = 1
R = np.diag([0.1, 0.1, 0.1, 0.1, 0.1])
#####################

dt = 0.01
nx = 6
ny = 5

phi, p, theta, q, psi, r = sympy.symbols('phi, p, theta, q, psi, r')
f = sympy.Matrix([
    p + sympy.sin(phi)*sympy.tan(theta)*q + sympy.cos(phi)*sympy.tan(theta)*r,
    0.0,
    sympy.cos(phi)*q - sympy.sin(phi)*r,
    0.0,
    sympy.sin(phi)*sympy.sec(theta)*q + sympy.cos(phi)*sympy.sec(theta)*r,
    0.0
])

A = f.jacobian([phi, p, theta, q, psi, r])
A_func = sympy.lambdify([phi, p, theta, q, psi, r], A, modules=['numpy', {'sec':sympy.sec}])
f = sympy.lambdify([phi, p, theta, q, psi, r], A, modules=['numpy', {'sec':sympy.sec}])

h = sympy.Matrix([phi, p, theta, q, r])
h_func = sympy.lambdify([phi, p, theta, q, psi, r], h, modules=['numpy', {'sec':sympy.sec}])

C = h.jacobian([phi, p, theta, q, psi, r])
C_func = sympy.lambdify([phi, p, theta, q, psi, r], C, modules=['numpy', {'sec':sympy.sec}])


x = sympy.Matrix([phi, p, theta, q, psi, r])
x0 = np.array([[0.0,0.0,0.0,0.0,0.0,0.0]]).T

V = Qc * np.array([[1/3*dt**3,1/2*dt**2,0,0,0,0],
                    [1/2*dt**2, dt, 0, 0,0,0],
                    [0,0,1/3*dt**3,1/2*dt**2,0,0],
                    [0,0,1/2*dt**2, dt,0,0],
                    [0,0,0,0,1/3*dt**3,1/2*dt**2],
                    [0,0,0,0,1/2*dt**2, dt]])

Q0 = V




imu = lab2_setup.IMU()




AttributeError: module 'esp32imu' has no attribute 'SerialDriver'

In [70]:
xh = np.empty((nx,1,10000))
Qh = np.empty((nx,nx,10000))

i = 0
xkk = x0
print(xkk.shape)
Qkk = Q0
xh[:,:,i] = x0
Qh[:,:,i] = Q0




while i<10000:
    i += 1
    # propogate statistics
    xkkm1 = xkk + dt*(A_func(xkk[0,0],xkk[1,0],xkk[2,0],xkk[3,0],xkk[4,0],xkk[5,0])@xkk)
    Qkkm1 = Qkk + dt*(A_func(xkk[0,0],xkk[1,0],xkk[2,0],xkk[3,0],xkk[4,0],xkk[5,0])@Qkk
                        + Qkk@A_func(xkk[0,0],xkk[1,0],xkk[2,0],xkk[3,0],xkk[4,0],xkk[5,0]).T
                        + V)
    Qkkm1 = Qkkm1.astype(np.float64)

    # get measurement
    y = np.zeros((5,1))
    acc = imu.get_acc()
    gyr = imu.get_gyr()
    y[0] = acc[0]
    y[2] = acc[1]
    y[1] = gyr[0]
    y[3] = gyr[1]
    y[4] = gyr[2]

    # update estimate using measurement
    Ck = C_func(xkk[0,0],xkk[1,0],xkk[2,0],xkk[3,0],xkk[4,0],xkk[5,0]).astype(np.float64)
    Lk = Qkkm1@Ck.T@np.linalg.inv(Ck@Qkkm1@Ck.T + R)
    xkk = xkkm1 + Lk@(y - h_func(xkk[0,0],xkk[1,0],xkk[2,0],xkk[3,0],xkk[4,0],xkk[5,0]))
    Qkk = (np.eye(nx) - Lk@Ck)@Qkkm1

    xh[:,:,i] = xkk
    Qh[:,:,i] = Qkk
    

(6, 1)


NameError: name 'imu' is not defined